In [1]:
import pandas as pd
import numpy as np
import os

In [39]:
# actual data
data_fetch_dates = ['0905', '0706', '0207']
actual_data = pd.concat([pd.merge(
    pd.read_csv(f'actualdata_trips_{date}.csv'), pd.read_csv(f'actualdata_stop_times_{date}.csv'), 
    how='left', left_on=['TripId'], right_on='TripId'
) for date in data_fetch_dates], axis=0)
actual_data.drop('EntityId', axis=1, inplace=True)
actual_data.set_index(['TripId', 'StartDate'], inplace=True)
actual_data.head()

,,RouteId,StartTime,StopId,StopSequence,ArrivalDelay,ArrivalTime,DepartureDelay,DepartureTime,ScheduleRelationship
TripId,StartDate,,,,,,,,,
254163638,20230508,43975,19:44:00,9057862,NaN,NaN,NaN,0.0,NaN,Scheduled
282351984,20230509,43245,17:12:00,9013478,NaN,NaN,NaN,0.0,NaN,Scheduled
282385362,20230508,43195,20:03:00,9058008,5.0,-60.0,NaN,0.0,NaN,Scheduled
264505093,20230508,59935,19:12:00,9050640,11.0,30.0,NaN,30.0,NaN,Scheduled
282384857,20230509,43216,04:37:00,9049320,NaN,NaN,NaN,0.0,NaN,Scheduled


In [40]:
#actual_data['trips']['id'] = [
#    f'{trip_id}{date}' for trip_id, date in zip(actual_data['trips']['TripId'], actual_data['trips']['StartDate'])
#]
#actual_data['trips'].set_index('id', inplace=True)
#actual_data['trips'].head(3)

In [5]:
# target data
dir_target = 'C:\\Users\\nstan\\OneDrive\\Winfo\\SoSe_23\\Data Integration\\target_data_vbn_modified'
rnames = ['trips', 'stops', 'stop_times', 'agency', 'routes', 'transfers']
target_data = {rname: pd.read_csv(f'{dir_target}\\{rname}.csv', low_memory=False) for rname in rnames}

In [ ]:
# step 1: join actual trips and stop_times
r1 = pd.merge(actual_data['stop_times'], actual_data['trips'], how='left', left_on=['TripId'], right_on=['TripId'])
r1

In [ ]:
# step 2a: join routes and agency (target data)
r2 = pd.merge(target_data['routes'], target_data['agency'], how='left', left_on=['agency_id'], right_on=['agency_id'])
r2.head(3)

In [ ]:
# step 2b: join the two relations using the Route Id
r3 = pd.merge(r1, r2, how='left', left_on=['RouteId'], right_on=['route_id'])
r3.head(3)

# finally, it is possible to...
# 1. determine the delay according to different agencies
# 2. determine the delay according to means of transportation

In [ ]:
# step 3: join TARGET trips and stop_times
r4 = pd.merge(target_data['stop_times'], target_data['trips'], how='left', left_on=['trip_id'], right_on=['trip_id'])
r4.head(3)

In [ ]:
# step 4: join r1 and r4 on Trip Id and Stop Id
# their schemata match for the most part
# however, using a left join enables to get all delays even if there is no join match
# (possible because there is no inclusion dependency)
r5 = pd.merge(r1, r4, how='inner', left_on=['TripId', 'StopId'], right_on=['trip_id', 'stop_id'])
r5